In [6]:
import pandas as pd
import requests
import datetime as dt
import json
import sys
import os
import sqlite3 as db

In [7]:
ebird_token = 'j6c7l80ga2ib'
db_name = 'trailheadDirectBirds_sous.db'

In [8]:
##connect to database
def connectDB():
    try:
        cnx = db.connect(db_name)
    except Exception as cnxError:
        raise UserWarning(f'Unable to connect to database due to: {cnxError}')
    return cnx

In [41]:
def get_hotspots():
    trailheadHotspots = pd.DataFrame()
    histHotObs = pd.DataFrame()
    cnx = connectDB()
    try:
        #get the latest hotspots from the database
        #trailheadHotspots = pd.read_sql('select * from Hotspots',con=cnx,parse_dates=['latestObsDt','latestUpdate'])   #Excluding Discovery Park until a later date
        trailheadHotspots = pd.read_sql('select * from Hotspots where Hotspots.StopName not like "DiscoveryPark%"',con=cnx,parse_dates=['latestObsDt','latestUpdate'])
        trailheadHotspots.set_index('index',inplace=True)
        trailheadHotspots.sort_values(by=['locId','latestObsDt'],ascending=False,ignore_index=True,inplace=True)
        uniqueHotspots = trailheadHotspots[['locId','latestObsDt']].drop_duplicates(subset=['locId'],keep='first')
        #get hotspots from the observations in the database
        histHotObs = pd.read_sql('select locId,obsDt,loadDate from historicObservations',con=cnx,parse_dates=['obsDt','loadDate'])
        min_obsDt = min(histHotObs['obsDt'])
        histHotObs.sort_values(by=['locId','obsDt'],ascending=False,ignore_index=True,inplace=True)
        histHotObs.drop_duplicates(subset=['locId'],keep='first',inplace=True)
        #merge the tables
        hotspots = pd.merge(left=uniqueHotspots,right=histHotObs,how='left',left_on='locId',right_on='locId')
        #remove hotspots without a recent checklist
        agingMask = hotspots.apply(lambda y: bool(y.loadDate is not(pd.NaT) or bool(y.latestObsDt < y.loadDate)),axis=1)
        hotspots = hotspots[~agingMask]
        #replace null, NaT observation date values with the earliest value found in the table
        hotspots.loc[(hotspots['obsDt'].isna()) | (hotspots['obsDt'].isnull()) | (hotspots['obsDt'] == ''),'obsDt'] = min_obsDt
        #recency check
        for x in hotspots:
            dateDiff = hotspots['latestObsDt']-hotspots['obsDt']
            hotspots['dateDiff'] = dateDiff.dt.days
        ##avoiding the big gaps for now....
        hotspots['run'] = hotspots['dateDiff'].between(7,1500,inclusive='both')
        hotspots = hotspots[hotspots['run'] == True]
        dfk = hotspots[['locId','latestObsDt','obsDt','dateDiff']]
    except Exception as ee:
        raise UserWarning(ee)
    finally: cnx.close()
    return dfk

In [33]:
##TODO #23 recency refactor: pare down results of hotspot batches to match known Trailhead Direct eBird hotspots

In [14]:
def get_histObs():
    import time
    from datetime import timedelta
    ObsHist = []
    cnx = connectDB()
    try:
        dfk_test = pd.DataFrame(get_hotspots().head(10))
        if dfk_test.empty:
            raise Exception('The provided dataframe is empty. There are either no eBird hotspots with new checklists or an error has occurred in the evaluation of the get_hotposts() function.')
        for locId in dfk_test.itertuples():
            startDate = locId.obsDt + timedelta(days=1)
            for x in pd.date_range(start=startDate,end=locId.latestObsDt,freq='D'):
                time.sleep(0.5)
                ymd = '{}/{}/{}'.format(x.year,x.month,x.day)
                ebird_baseUrl = 'https://api.ebird.org/v2/data/obs/'
                ebird_url = ebird_baseUrl + locId.locId + '/historic/' + ymd
                ebird_auth = {'X-eBirdApiToken': ebird_token}
                ebird_params = {
                    'fmt':'json',
                    'detail':'simple'
                }
                ebird_request = requests.get(ebird_url,params=ebird_params,headers=ebird_auth)
                if ebird_request.status_code == requests.codes.ok:
                    ebird_resp = pd.DataFrame(ebird_request.json())
                    ebird_resp['loadDate'] = dt.datetime.today()
                    ObsHist.append(ebird_resp)
                ebird_request.raise_for_status()
        historicObservations = pd.concat(ObsHist)
        historicObservations.to_sql(name='historicObservations_cooking',con=cnx,if_exists='append')
        cnx.close()
    except Exception as exc:
        raise exc
    return historicObservations

In [36]:
tgr = get_histObs()
tgr.head()

,loadDate,speciesCode,comName,sciName,locId,locName,obsDt,howMany,lat,lng,obsValid,obsReviewed,locationPrivate,subId
0,2022-03-03 19:20:26.285692,evegro,Evening Grosbeak,Coccothraustes vespertinus,L5915435,"stakeout Hooded Oriole, East Renton Highlands ...",2021-04-19 18:15,2.0,47.481462,-122.086774,True,False,False,S85936613
0,2022-03-03 19:32:11.371405,mallar3,Mallard,Anas platyrhynchos,L10129002,Salmon Run Nature Park,2021-11-24 11:42,5.0,47.539272,-122.043434,True,False,False,S98017243
1,2022-03-03 19:32:11.371405,hoomer,Hooded Merganser,Lophodytes cucullatus,L10129002,Salmon Run Nature Park,2021-11-24 11:42,1.0,47.539272,-122.043434,True,False,False,S98017243
2,2022-03-03 19:32:11.371405,commer,Common Merganser,Mergus merganser,L10129002,Salmon Run Nature Park,2021-11-24 11:42,3.0,47.539272,-122.043434,True,False,False,S98017243
3,2022-03-03 19:32:11.371405,belkin1,Belted Kingfisher,Megaceryle alcyon,L10129002,Salmon Run Nature Park,2021-11-24 11:42,1.0,47.539272,-122.043434,True,False,False,S98017243
